In [5]:
%matplotlib inline

# Imports & Packages

In [6]:
!pip install -U ipywidgets

In [7]:
import os
import re

import numpy as np
import pandas as pd
import ray
import torch
import torch.nn.functional as F
import torch.optim as optim
from ray import train as ray_train
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, random_split

# Constants

In [8]:
# Define Constants

DATA_PATH = "/kaggle/input/deep-learning-for-msc-202324/"
labels_train_path = DATA_PATH + "labels_train.csv"
sample_path = DATA_PATH + "sample.csv"
seqs_test_path = DATA_PATH + "seqs_test.csv"
seqs_train_path = DATA_PATH + "seqs_train.csv"
train_path = DATA_PATH + "train"
test_path = DATA_PATH + "test"

# Define a mapping from amino acid characters to integers
amino_acid_mapping = {
    'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4,
    'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9,
    'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14,
    'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19,
    'X': 20,  # Typically used for unknown amino acids
    'B': 21,  # Asparagine or Aspartic acid
    'Z': 22,  # Glutamine or Glutamic acid
    'J': 23,  # Leucine or Isoleucine
    '-': 24,  # Gap or padding
}

sec_struct_mapping = {'H': 0, 'E': 1, 'C': 2}

# Dataset Class & Utils

In [9]:
class ProteinDataset(Dataset):
    def __init__(self, csv_file, train_dir, label_file=None, normalize_method='min-max'):

        # Load the sequences
        self.seqs = pd.read_csv(csv_file)

        # Load the protein data from the directory
        self.protein_data = {}
        for filename in os.listdir(train_dir):
            if filename.endswith(".csv"):  # Check if the file is a CSV
                protein_id = re.split(r'_train|_test', filename)[0]
                self.protein_data[protein_id] = pd.read_csv(os.path.join(train_dir, filename))

        # Load the labels, if provided
        if label_file:
            self.labels = pd.read_csv(label_file)
        else:
            self.labels = None

        # Amino acid mapping
        self.amino_acid_mapping = amino_acid_mapping
        self.normalize_method = normalize_method

    def encode_sequence(self, sequence):
        # Convert each amino acid in the sequence to a one-hot encoded vector
        encoded_sequence = np.zeros((len(sequence), len(self.amino_acid_mapping)), dtype=int)
        for i, amino_acid in enumerate(sequence):
            # Default to 'X' for unknown amino acids
            index = self.amino_acid_mapping.get(amino_acid, self.amino_acid_mapping['X'])
            encoded_sequence[i, index] = 1
        return encoded_sequence

    def normalize_pssm(self, pssm):
        # Assuming the first two columns are non-numeric; adjust as necessary based on your actual data format
        numeric_columns = pssm[:, 2:]  # Adjust this if your numeric data starts from a different column

        # Convert to floats
        try:
            pssm_numeric = numeric_columns.astype(np.float32)
        except ValueError as e:
            # Handle or log the error if needed
            raise ValueError(f"Error converting PSSM to float: {e}")

        if self.normalize_method == 'min-max':
            # Min-Max normalization
            pssm_min = pssm_numeric.min(axis=0)
            pssm_max = pssm_numeric.max(axis=0)
            # Ensure no division by zero
            pssm_range = np.where(pssm_max - pssm_min == 0, 1, pssm_max - pssm_min)
            normalized_pssm = (pssm_numeric - pssm_min) / pssm_range
        elif self.normalize_method == 'z-score':
            # Z-Score normalization
            pssm_mean = pssm_numeric.mean(axis=0)
            pssm_std = pssm_numeric.std(axis=0)
            # Avoid division by zero
            pssm_std = np.where(pssm_std == 0, 1, pssm_std)
            normalized_pssm = (pssm_numeric - pssm_mean) / pssm_std
        else:
            # If no normalization method provided, return the original PSSM
            normalized_pssm = pssm_numeric

        return normalized_pssm

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        protein_id = self.seqs.iloc[idx]['PDB_ID']
        sequence = self.seqs.iloc[idx]['SEQUENCE']
        encoded_sequence = self.encode_sequence(sequence)  # Encode the sequence
        pssm = self.protein_data[protein_id].values  # Assuming you will process PSSM separately
        normalized_pssm = self.normalize_pssm(pssm)  # Ensure this is uncommented to use normalized PSSM

        if self.labels is not None:
            label_seq = self.labels.iloc[idx]['SEC_STRUCT']
            label_numeric = [sec_struct_mapping[char] for char in label_seq]
            label_tensor = torch.tensor(label_numeric, dtype=torch.long)
            return (
                protein_id,
                torch.tensor(encoded_sequence, dtype=torch.float32),
                torch.tensor(normalized_pssm, dtype=torch.float32),
                label_tensor
            )

        return (
            protein_id,
            torch.tensor(encoded_sequence, dtype=torch.float32),
            torch.tensor(normalized_pssm, dtype=torch.float32)
        )



In [10]:
def collate_fn(batch):
    _, sequences, pssms, labels_list = zip(*batch)  # Unzip the batch

    # Pad sequences and PSSMs
    sequences_padded = pad_sequence([seq.clone().detach() for seq in sequences], batch_first=True)

    pssms_padded = pad_sequence([pssm.clone().detach() for pssm in pssms], batch_first=True)

    # Handling labels correctly
    if labels_list[0] is not None:  # Check if labels exist
        labels_padded = pad_sequence([label.clone().detach() for label in labels_list], batch_first=True)

    else:
        labels_padded = None

    # Create a mask based on the original sequence lengths
    mask = [torch.ones(len(label), dtype=torch.uint8) for label in labels_list]
    mask_padded = pad_sequence(mask, batch_first=True, padding_value=0)  # Assuming padding_value for labels is 0
    return sequences_padded, pssms_padded, labels_padded, mask_padded


# Fully Convolutional Networks (FCNs)

In [11]:
class FullyConvolutionalProteinModel(nn.Module):
    def __init__(
            self, num_classes=3,
            input_channels=20,
            hidden_layers_number=3,
            dropout_rate=0.233246
    ):
        super(FullyConvolutionalProteinModel, self).__init__()

        self.hidden_layers = self.get_hidden_layers_size(
            hidden_layers_number)  # List of out_channels for each hidden layer
        self.dropout_rate = dropout_rate

        # Creating convolutional layers dynamically based on 'hidden_layers' input
        self.convs = nn.ModuleList()
        in_channels = input_channels
        for out_channels in self.hidden_layers:
            self.convs.append(
                nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
            )
            in_channels = out_channels  # Next layer's in_channels is current layer's out_channels

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Final layer that maps to the number of classes
        # The last item of hidden_layers list is used as in_channels here
        self.final_conv = nn.Conv1d(in_channels=self.hidden_layers[-1], out_channels=num_classes, kernel_size=1)

    def forward(self, x):
        # Apply convolutional layers with activation functions and dropout
        for conv in self.convs:
            x = F.relu(conv(x))
            x = self.dropout(x)  # Apply dropout after activation

        # Apply final convolutional layer - no activation, as CrossEntropyLoss includes it
        x = self.final_conv(x)

        # No softmax here, as nn.CrossEntropyLoss applies it internally.
        # Transpose the output to match [batch_size, sequence_length, num_classes]
        x = x.transpose(1, 2)

        return x

    def get_hidden_layers_size(self, number):
        hidden_layers_configs = {
            1: [64],  # one layer with 64 channels
            2: [64, 128],  # two layers with 64 and 128 channels
            3: [64, 128, 256],  # three layers
            4: [64, 128, 256, 512],  # four layers
            5: [64, 128, 256, 512, 1024]  # five layers
        }
        return hidden_layers_configs[number]

# Train, Validate & Test

In [12]:
def train_model(model, criterion, optimizer, train_dataloader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for sequences, pssms, labels, _ in train_dataloader:
            inputs = pssms.permute(0, 2, 1)  # Adjust for PSSM data

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.transpose(1, 2), labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            # Calculate training accuracy
            _, predicted = torch.max(outputs, 2)  # Get the index of the max log-probability
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.numel()

        epoch_loss = running_loss / len(train_dataloader.dataset)
        epoch_acc = correct_preds / total_preds
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

In [13]:
def validate_model(model, criterion, val_dataloader):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for sequences, pssms, labels, _ in val_dataloader:
            inputs = pssms.permute(0, 2, 1)  # Adjust for PSSM data

            outputs = model(inputs)
            loss = criterion(outputs.transpose(1, 2), labels)

            running_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            _, predicted = torch.max(outputs, 2)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.numel()

    val_loss = running_loss / len(val_dataloader.dataset)
    val_acc = correct_preds / total_preds
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

In [14]:
def test_model_direct(model, test_dataset, output_file='submission.csv'):
    model.eval()  # Set the model to evaluation mode
    predictions = []

    with torch.no_grad():
        for i in range(len(test_dataset)):  # Iterate directly over the dataset
            pdb_id, _, pssm = test_dataset[i]  # Assuming the dataset returns PDB_ID, sequence, and PSSM

            # Prepare the input tensor; add an extra batch dimension using unsqueeze
            input_pssm = pssm.unsqueeze(0).permute(0, 2, 1)  # Adjust dimensions to [1, features, seq_len]

            # Make a prediction
            outputs = model(input_pssm)
            _, predicted = torch.max(outputs, 2)  # Get the index of max log-probability

            # Process the predictions
            seq_len = pssm.shape[0]  # Assuming pssm is [features, seq_len]
            for j in range(seq_len):
                residue_id = f"{pdb_id}_{j + 1}"  # Construct the ID
                structure_label = ['H', 'E', 'C'][predicted[0, j].item()]  # Map numeric predictions to labels
                predictions.append([residue_id, structure_label])

    # Write predictions to CSV
    pd.DataFrame(predictions, columns=['ID', 'STRUCTURE']).to_csv(output_file, index=False)
    print(f'Submission file saved to {output_file}')

In [15]:
def get_optimizer(optimizer_type, model, lr, weight_decay):
    # Choose the optimizer based on the parameterization
    if optimizer_type == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_type == "rmsprop":
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError("Unknown optimizer")

    return optimizer


def train_with_params(
        lr=0.001,
        batch_size=4,
        hidden_layers=5,
        dropout_rate=0.233246,
        weight_decay=0.0,
        optimizer='rmsprop',
        normalization='min-max',
        num_epochs=10,
        output_file='submission.csv'
):
    train_dataset = ProteinDataset(csv_file=seqs_train_path, train_dir=train_path, label_file=labels_train_path,
                                   normalize_method=normalization)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    test_dataset = ProteinDataset(csv_file=seqs_test_path, train_dir=test_path, normalize_method=normalization)

    # Splitting train_dataset into train and validation sets (adjust sizes as needed)
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    model = FullyConvolutionalProteinModel(hidden_layers_number=hidden_layers, dropout_rate=dropout_rate)
    criterion = nn.CrossEntropyLoss()
    optimizer = get_optimizer(optimizer, model, lr, weight_decay)

    train_model(model, criterion, optimizer, train_dataloader, num_epochs)
    validate_model(model, criterion, val_loader)
    test_model_direct(model, test_dataset, output_file)


# Main

# Hyperparameter Tuning: ax_client

In [16]:
!pip install ax-platform

In [17]:
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

import os
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch._tensor import Tensor
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.service.utils.report_utils import exp_to_df
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.utils.tutorials.cnn_utils import evaluate, load_mnist, train



def train_evaluate(parameterization):
    # Initialize your model with hyperparameters
    
    # This is an example mapping, adjust according to your needs
    hidden_layers_configs = {
        1: [64],  # one layer with 64 channels
        2: [64, 128],  # two layers with 64 and 128 channels
        3: [64, 128, 256],  # three layers
        4: [64, 128, 256, 512],  # four layers
        5: [64, 128, 256, 512, 1024]  # five layers
    }

    # Inside your train_evaluate function
    hidden_layers_structure = hidden_layers_configs[parameterization.get("hidden_layers", 2)]

    model = FullyConvolutionalProteinModel(
        num_classes=3, 
        input_channels=20,
        hidden_layers=hidden_layers_structure,  # Default to 2 if not specified
        dropout_rate=parameterization.get("dropout_rate", 0.5)  # Default to 0.5 if not specified
    )
    criterion = nn.CrossEntropyLoss()
    
    # Choose the optimizer based on the parameterization
    if parameterization["optimizer"] == "adam":
        optimizer = optim.Adam(model.parameters(), lr=parameterization["lr"], weight_decay=parameterization.get("weight_decay", 0))
    elif parameterization["optimizer"] == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=parameterization["lr"], momentum=0.9, weight_decay=parameterization.get("weight_decay", 0))
    elif parameterization["optimizer"] == "rmsprop":
        optimizer = optim.RMSprop(model.parameters(), lr=parameterization["lr"], weight_decay=parameterization.get("weight_decay", 0))
    else:
        raise ValueError("Unknown optimizer")
        
        
    dataset = ProteinDataset(csv_file=seqs_train_path, train_dir=train_path, label_file=labels_train_path, normalize_method=parameterization["normalization"])
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_subset, val_subset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_subset, batch_size=int(parameterization["batch_size"]), shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_subset, batch_size=int(parameterization["batch_size"]), shuffle=False, collate_fn=collate_fn)
    
    # Your training and validation loop
    for epoch in range(parameterization.get("epochs", 10)):  # Use the epochs parameter, default to 10
        train_model(model, criterion, optimizer, train_loader)  # Assume implementation
        val_loss, val_acc = validate_model(model, criterion, val_loader)  # Assume implementation
    
    return {"accuracy": val_acc, "loss": -val_loss}

# Set up the Ax experiment
ax_client = AxClient(enforce_sequential_optimization=False)
ax_client.create_experiment(
    name="protein_model_experiment",
    parameters=[
        {"name": "lr", "type": "range", "bounds": [0.0001, 0.01], "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [16, 32, 64, 128]},
        {"name": "hidden_layers", "type": "choice", "values": [2, 3, 4, 5]},
        {"name": "dropout_rate", "type": "range", "bounds": [0.0, 0.7]},
        {"name": "weight_decay", "type": "range", "bounds": [0.0, 0.1]},
        {"name": "epochs", "type": "fixed", "value": 10},  # Fixed for all trials, can be changed as needed
        {"name": "optimizer", "type": "choice", "values": ["adam", "sgd", "rmsprop"]},  # Add optimizer as a choice
        {"name": "normalization", "type": "choice", "values": ["min-max", "z-score"]},  # Add optimizer as a choice

    ],
    objectives={"accuracy": ObjectiveProperties(minimize=False)}
)

# Running the trials
for i in range(30):  # Number of iterations
    params, trial_index = ax_client.get_next_trial()
    metrics = train_evaluate(params)
    ax_client.complete_trial(trial_index=trial_index, raw_data=metrics)

# Fetch the best parameters
best_parameters, metrics = ax_client.get_best_parameters()
best_metrics = metrics['objectives']
print(f'Best Parameters: {best_parameters}')
print(f'Best Accuracy: {best_metrics}')


[INFO 03-14 01:24:40] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-14 01:24:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-14 01:24:40] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter batch_size. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/home/alex/anaconda3/lib/python3.11/site-packages/ax/core/parameter.py:518: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "batch_size". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify 

TypeError: FullyConvolutionalProteinModel.__init__() got an unexpected keyword argument 'hidden_layers'

# Feature Selection

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder

def load_data_in_batches(seqs_train_path, train_path, labels_train_path, batch_size=100):
    # Load sequence and label information
    seqs_df = pd.read_csv(seqs_train_path)
    labels_df = pd.read_csv(labels_train_path)
    labels_map = dict(zip(labels_df['PDB_ID'], labels_df['SEC_STRUCT']))

    batch_pssms = []
    batch_labels = []

    # Process in batches
    for batch_start in range(0, len(seqs_df), batch_size):
        for _, row in seqs_df.iloc[batch_start:batch_start + batch_size].iterrows():
            pdb_id = row['PDB_ID']
            seq_label = labels_map.get(pdb_id, None)

            if seq_label:
                pssm_file = os.path.join(train_path, f"{pdb_id}_train.csv")
                if os.path.exists(pssm_file):
                    pssm_df = pd.read_csv(pssm_file, usecols=range(2, 22))
                    pssm_values = pssm_df.to_numpy()
                    for pssm_row in pssm_values:
                        batch_pssms.append(pssm_row)
                        batch_labels.append(seq_label)  # Assuming one label per sequence

        # Perform feature selection on the batch
        encoded_labels = LabelEncoder().fit_transform(batch_labels)
        f_values, p_values = f_classif(batch_pssms, encoded_labels)
        
        # Log or store batch results for later analysis
        print(f"Batch {batch_start // batch_size + 1}")
        for i in range(len(f_values)):
            print(f"Feature {i}: F-value = {f_values[i]:.3f}, p-value = {p_values[i]:.3f}")

        # Reset for next batch
        batch_pssms = []
        batch_labels = []

# Usage
DATA_PATH = "/kaggle/input/deep-learning-for-msc-202324/"
load_data_in_batches(DATA_PATH + 'seqs_train.csv', DATA_PATH + 'train', DATA_PATH + 'labels_train.csv', batch_size=100)
